In [1]:
!pip install MTCNN

     |████████████████████████████████| 2.3 MB 13.7 MB/s 


In [2]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from google.colab.patches import cv2_imshow
import math

In [3]:
def pixelated(img):
  (h,w)=img.shape[:2]
  X = np.linspace(0,w,5,dtype='int')
  Y = np.linspace(0,h,5,dtype='int')

  for i in range(1,len(Y)):
    for j in range(1,len(X)):
      roi=img[Y[i-1]:Y[i],X[j-1]:X[j]]
      (B,G,R)=[int(x) for x in cv2.mean(roi)[:3]]
      cv2.rectangle(img,(X[j-1],Y[i-1]),(X[j],Y[i]),(B,G,R),-1)

  return img


In [5]:
vid=cv2.VideoCapture("/content/sample.mp4")
if vid.isOpened()==False:
  print("File not opening")


while (vid.isOpened()):
  ret,frame=vid.read()
  resized=cv2.resize(frame,(256,128),interpolation=cv2.INTER_AREA)
  detector=MTCNN()
  faces=detector.detect_faces(resized)
  if faces!=[]:
    # print('face detected')
    for x in faces:
      if x['confidence']>=0.9:
        bounding_box=x['box']
        coordinates=x['keypoints']
        # cv2.rectangle(resized,(bounding_box[0], bounding_box[1]),(bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),(255,155,200),1)
        roi = resized[x['box'][1]:x['box'][1]+x['box'][3],x['box'][0]:x['box'][0]+x['box'][2]]
        blurred = pixelated(roi)
        resized[x['box'][1]:x['box'][1]+x['box'][3],x['box'][0]:x['box'][0]+x['box'][2]]=blurred

  cv2_imshow(resized) # if using colab otherwise comment it
  # cv2.imshow('window',resized) # if using jupyter otherwise comment it
  



  if cv2.waitKey(1) & 0xFF==ord('q'):
    break


vid.release()
cv2.destroyAllWindows()